# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
G. Rouille  ->  G. Rouille  |  ['G. Rouille']
Arxiv has 35 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2401.00934


extracting tarball to tmp_2401.00934...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


Found 146 bibliographic references in tmp_2401.00934/main.bbl.
syntax error in line 396: '=' expected
Retrieving document from  https://arxiv.org/e-print/2401.01221


extracting tarball to tmp_2401.01221...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2401.01221/aa.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'authors' from 'tmp_2401.01221/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.00934-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.00934) | **JADES: Rest-frame UV-to-NIR Size Evolution of Massive Quiescent Galaxies  from Redshift z=5 to z=0.5**  |
|| Z. Ji, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-01-03*|
|*Comments*| *28 pages, 19 figures, submitted to ApJ*|
|**Abstract**| We present the UV-to-NIR size evolution of a sample of 161 quiescent galaxies (QGs) with $M_*>10^{10}M_\odot$ over $0.5<z<5$. With deep multi-band NIRCam images in GOODS-South from JADES, we measure the effective radii ($R_e$) of the galaxies at rest-frame 0.3, 0.5 and 1$\mu m$. On average, QGs are 45% (15%) more compact at rest-frame 1$\mu m$ than they are at 0.3$\mu m$ (0.5$\mu m$). Regardless of wavelengths, the $R_e$ of QGs strongly evolves with redshift, and this evolution depends on stellar mass. For lower-mass QGs with $M_*=10^{10}-10^{10.6}M_\odot$, the evolution follows $R_e\sim(1+z)^{-1.1}$, whereas it becomes steeper, following $R_e\sim(1+z)^{-1.7}$, for higher-mass QGs with $M_*>10^{10.6}M_\odot$. To constrain the physical mechanisms driving the apparent size evolution, we study the relationship between $R_e$ and the formation redshift ($z_{form}$) of QGs. For lower-mass QGs, this relationship is broadly consistent with $R_e\sim(1+z_{form})^{-1}$, in line with the expectation of the progenitor effect. For higher-mass QGs, the relationship between $R_e$ and $z_{form}$ depends on stellar age. Older QGs have a steeper relationship between $R_e$ and $z_{form}$ than that expected from the progenitor effect alone, suggesting that mergers and/or post-quenching continuous gas accretion drive additional size growth in very massive systems. We find that the $z>3$ QGs in our sample are very compact, with mass surface densities $\Sigma_e\gtrsim10^{10} M_\odot/\rm{kpc}^2$, and their $R_e$ are possibly even smaller than anticipated from the size evolution measured for lower-redshift QGs. Finally, we take a close look at the structure of GS-9209, one of the earliest confirmed massive QGs at $z_{spec}\sim4.7$. From UV to NIR, GS-9209 becomes increasingly compact, and its light profile becomes more spheroidal, showing that the color gradient is already present in this earliest massive QG. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.01221-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.01221) | **PDRs4All. V. Modelling the dust evolution across the illuminated edge of  the Orion Bar**  |
|| M. Elyajouri, et al. -- incl., <mark>G. Rouille</mark> |
|*Appeared on*| *2024-01-03*|
|*Comments*| *18 pages, 11 figures, accepted for publication in A&A*|
|**Abstract**| We study the emission of dust grains within the Orion Bar - a well-known, highly far-UV (FUV)-irradiated PDR. The Orion Bar because of its edge-on geometry provides an exceptional benchmark for characterizing dust evolution and the associated driving processes under varying physical conditions. Our goal is to constrain the local properties of dust by comparing its emission to models. Taking advantage of the recent JWST PDRs4All data, we follow the dust emission as traced by JWST NIRCam (at 3.35 and 4.8 micron) and MIRI (at 7.7, 11.3, 15.0, and 25.5 micron), along with NIRSpec and MRS spectroscopic observations. First, we constrain the minimum size and hydrogen content of carbon nano-grains from a comparison between the observed dust emission spectra and the predictions of the THEMIS dust model coupled to the numerical code DustEM. Using this dust model, we then perform 3D radiative transfer simulations of dust emission with the SOC code and compare to data obtained along well chosen profiles across the Orion Bar. The JWST data allows us, for the first time, to spatially resolve the steep variation of dust emission at the illuminated edge of the Orion Bar PDR. By considering a dust model with carbonaceous nano-grains and submicronic coated silicate grains, we derive unprecedented constraints on the properties of across the Orion Bar. To explain the observed emission profiles with our simulations, we find that the nano-grains must be strongly depleted with an abundance (relative to the gas) 15 times less than in the diffuse ISM. The NIRSpec and MRS spectroscopic observations reveal variations in the hydrogenation of the carbon nano-grains. The lowest hydrogenation levels are found in the vicinity of the illuminating stars suggesting photo-processing while more hydrogenated nano-grains are found in the cold and dense molecular region, potentially indicative of larger grains. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2401.00934.md
    + _build/html/tmp_2401.00934/./size_evolution.png
    + _build/html/tmp_2401.00934/./filter_choice.png
    + _build/html/tmp_2401.00934/./size_comparison_zgt3.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\prospector}{{\sc Prospector}\xspace}$
$\newcommand{\tlookback}{{t_{\rm{lookback}}}\xspace}$
$\newcommand{\wren}[1]{\textcolor{teal}{#1}}$</div>



<div id="title">

# JADES: Rest-frame UV-to-NIR Size Evolution of Massive Quiescent Galaxies from Redshift $z=5$ to $z=0.5$ 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.00934-b31b1b.svg)](https://arxiv.org/abs/2401.00934)<mark>Appeared on: 2024-01-03</mark> -  _28 pages, 19 figures, submitted to ApJ_

</div>
<div id="authors">

Z. Ji, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** We present the UV-to-NIR size evolution of a sample of 161 quiescent galaxies with $M_* > 10^{10}M_\sun$ over $0.5<z<5$ . With deep multi-band NIRCam images in GOODS-South from JADES, we measure the effective radii ( $R_e$ ) of the galaxies at rest-frame 0.3, 0.5 and 1 $\micron$ . On average, we find that quiescent galaxies are 45 \% (15 \% ) more compact at rest-frame 1 $\micron$ than they are at 0.3 $\micron$ (0.5 $\micron$ ). Regardless of wavelengths, the $R_e$ of quiescent galaxies strongly evolves with redshift, and this evolution depends on stellar mass.  For lower-mass quiescent galaxies with $M_* = 10^{10}-10^{10.6}M_\sun$ , the evolution follows $R_e\propto(1+z)^{-1.1}$ , whereas it becomes steeper, following $R_e\propto(1+z)^{-1.7}$ , for higher-mass quiescent galaxies with $M_* > 10^{10.6}M_\sun$ . To constrain the physical mechanisms driving the apparent size evolution, we study the relationship between $R_e$ and the formation redshift ( $z_{\rm{form}}$ ) of quiescent galaxies. For lower-mass quiescent galaxies, this relationship is broadly consistent with $R_e\propto(1+z_{\rm{form}})^{-1}$ , in line with the expectation of the progenitor effect. For higher-mass quiescent galaxies, the relationship between $R_e$ and $z_{\rm{form}}$ depends on stellar age. Older quiescent galaxies have a steeper relationship between $R_e$ and $z_{\rm{form}}$ than that expected from the progenitor effect alone, suggesting that mergers and/or post-quenching continuous gas accretion drive additional size growth in very massive systems. We find that the $z>3$ quiescent galaxies in our sample are very compact, with mass surface densities $\Sigma_e \gtrsim 10^{10} M_\sun/\rm{kpc}^2$ , and their $R_e$ are possibly even smaller than anticipated from the size evolution measured for lower-redshift quiescent galaxies. Finally, we take a close look at the structure of GS-9209, one of the earliest confirmed massive quiescent galaxies at $z_{\rm{spec}} \sim 4.7$ . From UV to NIR, GS-9209 becomes increasingly compact, and its light profile becomes more spheroidal, showing that the color gradient is already present in this earliest massive quiescent galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2401.00934/./size_evolution.png" alt="Fig12" width="100%"/>

**Figure 12. -** Rest-frame size evolution of massive quiescent galaxies. The corresponding best-fit parameters can be found in Table \ref{tab:size_evo}. ** Top:** The left panel shows the best-fit size evolution at rest-frame 0.3$\micron$(blue), 0.5$\micron$(green) and 1$\micron$(red), respectively. The shaded regions mark the $1\sigma$ uncertainties of the best-fit relations. The filled squares with error bars are median $R_e$ and their uncertainties in individual redshift bins (Section \ref{sec:size_evo}). The right panel shows the 1-, 2- and 3-$\sigma$ contours of the $R_e$-z relations based on our bootstrap Monte Carlo method (Section \ref{sec:size_evo}). ** Bottom:** Differing from the top-left panel, here we plot individual quiescent galaxies to the size-evolution diagram. Like previous figures, GS-9209 is marked with the orange square. (*fig:size_evo*)

</div>
<div id="div_fig2">

<img src="tmp_2401.00934/./filter_choice.png" alt="Fig2" width="100%"/>

**Figure 2. -** Wide-band filters used for the rest-frame 0.3$\micron$(blue), 0.5$\micron$(green) and 1$\micron$(red) size measures. Expect the HST/ACS F606W, all other filters are from JWST/NIRCam. (*fig:filter_selection*)

</div>
<div id="div_fig3">

<img src="tmp_2401.00934/./size_comparison_zgt3.png" alt="Fig4" width="100%"/>

**Figure 4. -** Rest-frame 1$\micron$ sizes of the $z>3$ quiescent galaxies from different methods. The $x$-axis shows the fiducial measurements in this work (Section \ref{sec:galfit}). The $y$-axis shows the measurements from alternative methods detailed in Section \ref{sec:test_re_zgt3}, including using (1) a different morphological fitting tool {\sc Lenstronomy}(black circles), (2) {\sc Galfit} but fixing Sérsic index $n=1$(blue squares) or $n=4$(red squares) during the fit and (3) a nonparametric method with the Richardson–Lucy deconvolution (orange triangles). The main purpose here is to compare the sizes from different methodologies, we thus do not estimate uncertainties for the $y$-axis. The dashed line marks the one-to-one relation, and the dotted lines mark the 2 times above/below the one-to-one relation. For the vast majority of the $z>3$ quiescent galaxies, the relative difference in sizes from different methods is $<50\%$.  (*fig:size_zgt3*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.00934"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

348  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

1  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
